In [1]:
!pip install pipeline
!pip install openai
!pip install openai langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.6/377.6 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.4 MB/s eta 0:00:00


In [3]:
import json
import openai
import os
import re

In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

In [8]:
file_path = '/content/sample2.txt'

with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

# Method1

## 1. 5페이지만 입력하여 참여 팀원, 주제, 목차 내용 추출

### 첫 5페이지의 텍스트를 text5 변수에 저장

In [9]:
end_marker = text.find('<p.6>')

if end_marker != -1:
    text5 = text[:end_marker]
else:
    text5 = text

In [10]:
text5

'<p.1>\n2024 BITAmin 겨울 연합프로젝트 시계열 1조\nNetflix Stock Price Prediction with News Topic & Sentiment\n시계열 1조\n12기 송규헌\n12기 권도영\n12기 이태경\n13기 김서윤\n13기 한진솔\n\n<p.2>\nCONTENTS\n01. INTRODUCTION\n02. DATA PREPROCESSING\n03. MODELING\n04. CONCLUSIONS AND LIMITATION\n\n<p.3>\n01. INTRODUCTION\n\n<p.4>\n01. INTRODUCTION\n1.1 Background of topic selection\n1.뉴스가 주가 변동에 미치는 영향 탐구\n주가 예측에 뉴스를 활용할 수 있는지 탐구해보고자 함\n주가를 예측하는 데 사용하는 데이터로 뉴스의 감성분석 및 토픽 모델링 결과를 사용하고자 함\n뉴스 기사에서는 주로 한 기업에 대해 보도하고 있어 예측 대상은 한 개의 주식 종목으로 정함\n뉴스 감성 분석/토픽 모델링 결과를 활용하여 주가를 예측하는 프로젝트는 많지 않아, 직접 뉴스 데이터를 활용하고자 함\n\n<p.5>\n01. INTRODUCTION\n1.1 Background of topic selection\n2. 장기적인 추세를 고려할 수 있는 주가 예측 모델 구현\n기존의 주가 예측 프로젝트는 주로 모델의 예측 일수나 시퀀스 길이를 1~5로 설정함\n짧은 시간 동안의 주가 변동을 예측한 프로젝트와 달리 예측 일수와 시퀀스 길이를 늘려 단기적인 변동뿐만 아니라 장기적인 추세를 고려한 예측 결과를 얻고자 함\n기존 프로젝트에서 주가 예측 그래프가 실제 주가 그래프를 단순 평행 이동한 형태로 나타나는 문제를 해결하고자 함\n\n'

### 마크다운 형식으로 팀원, 주제, main topic 추출

In [4]:
api_key_filepath = "/content/openai_api_key.json"
with open(api_key_filepath, 'r') as f:
    api_key = json.load(f)
api_key = api_key['OPENAI_API_KEY']

In [6]:
os.environ['OPENAI_API_KEY'] = api_key

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    openai_api_key=os.environ['OPENAI_API_KEY']
)

In [11]:
def extract_information(text5):
    prompt = f"""
    Extract the following information from the provided text:
    1. Name (excluding the team name, include all people listed)
    2. Title (include the entire title as it appears in the text)
    3. Main Topics (only main topics from the Table of Contents, excluding subtopics)

    The Main Topics should be extracted from the section that follows headers such as 'TABLE OF CONTENTS', '목차 소개', or any similar variation.
    Ensure that main topics are not split into multiple lines. Main topics should be connected by commas if there are more than one.
    Ensure that team member's name are not split into multiple lines. Name should be connected by commas if there are more than one.
    Ensure that title is not cut off and is extracted in their entirety.
    Exclude any subtopics or secondary information while extracting main topics.
    Stop at the next page marker '<p.'.

    Do not include any additional notes or explanations in the output.

    Text: {text5}


    Format the extracted information as follows:
    <subject>title</subject>
    <team>team members</team>
    <index>main topics</index>

    """

    response = llm.invoke(prompt)
    extracted_info = response.content.strip()
    return extracted_info

extracted_info = extract_information(text5)
print(extracted_info)


<subject>Netflix Stock Price Prediction with News Topic & Sentiment</subject>
<team>12기 송규헌, 12기 권도영, 12기 이태경, 13기 김서윤, 13기 한진솔</team>
<index>INTRODUCTION, DATA PREPROCESSING, MODELING, CONCLUSIONS AND LIMITATION</index>


In [ ]:
main_topics_match = re.search(r'<index>(.*?)</index>', extracted_info, re.DOTALL)
if main_topics_match:
    main_topics = main_topics_match.group(1).strip()
else:
    main_topics = ""

## 전체 텍스트에서 대주제, 소주제, 핵심내용 추출

### 5페이지에서 추출한 대주제를 사용하는 경우

In [ ]:
def extract_detailed_information(text, main_topics):
    prompt = f"""
    Extract detailed information for each main topic and its subtopics from the provided text.

    Format the extracted information as follows:
    <main>main topic</main>
    <sub>subtopic</sub>
    <content>content</content>

    Use the main topics provided below:
    {main_topics}

    Do not include any additional notes or explanations in the output.

    Text: {text}
    """

    response = llm.invoke(prompt)
    extracted_details = response.content.strip()
    return extracted_details

In [ ]:
detailed_info = extract_detailed_information(text, main_topics)
print(detailed_info)

<main>INTRODUCTION</main>
<sub>Background of topic selection</sub>
<content>뉴스가 주가 변동에 미치는 영향 탐구 주가 예측에 뉴스를 활용할 수 있는지 탐구해보고자 함 주가를 예측하는 데 사용하는 데이터로 뉴스의 감성분석 및 토픽 모델링 결과를 사용하고자 함 뉴스 기사에서는 주로 한 기업에 대해 보도하고 있어 예측 대상은 한 개의 주식 종목으로 정함 뉴스 감성 분석/토픽 모델링 결과를 활용하여 주가를 예측하는 프로젝트는 많지 않아, 직접 뉴스 데이터를 활용하고자 함</content>

<sub>Brief Project Introduction</sub>
<content>[목표] 뉴스 데이터를 활용한 주가 예측 모델의 최적화 및 효율적인 파라미터 선정 [주요 내용] 장기적인 추세 예측에 효과적인 모델 판별: LSTM vs. GRU vs. Transformer 실험을 통한 모델 파라미터 튜닝 뉴스 감성분석과 토픽 모델링의 주가 예측 유용성 검토 각 실험 결과 평가에 필요한 지표 선정 [개발 환경] Colab</content>

<sub>Data collection</sub>
<content>FinanceDataReader 라이브러리를 활용하여 NETFLIX 기업의 2018년 1월 2일 ~ 2023년 12월 29일 주가 데이터 수집 동일한 기간에 대한 핀터레스트, 메타플렛폼스, 스포티파이의 주가 데이터 수집</content>

<main>DATA PREPROCESSING</main>
<sub>Make derived variable</sub>
<content>Roc(Range of Change) 변화율 : 1d_RoC, 5d_RoC MA(Moving Average) 이동 평균 : 5MA, 120MA 이전 시점을 포함하여 rolling하는 변수이므로 생성된 파생변수의 첫 시점은 NaN 상태. => 2018년 데이터를 추가로 수집해 파생 변수 생성에 활용</content>

<sub>Add indecators</s

## 요약

In [ ]:
def summarize_content(content):
    if not content.strip():
        return content

    prompt = f"""
    다음 내용을 요약해줘:

    내용: {content}

    위 내용을 간결하게 요약해줘.
    """

    response = llm.invoke(prompt)
    summary = response.content.strip()
    return summary

summarized_detailed_info = re.sub(
    r'(<content>)(.*?)(</content>)',
    lambda m: f"{m.group(1)}{summarize_content(m.group(2))}{m.group(3)}",
    detailed_info,
    flags=re.DOTALL
)

print(summarized_detailed_info)

<main>INTRODUCTION</main>
<sub>Background of topic selection</sub>
<content>뉴스의 감성분석과 토픽 모델링을 사용하여 주가를 예측하는 프로젝트를 진행하기 위해 뉴스 데이터를 활용하고자 함.</content>

<sub>Brief Project Introduction</sub>
<content>뉴스 데이터를 활용한 주가 예측 모델의 최적화와 효율적인 파라미터 선정을 위해 LSTM, GRU, Transformer 모델을 실험하고 파라미터를 튜닝하며, 뉴스 감성분석과 토픽 모델링의 주가 예측 유용성을 검토한다. Colab을 개발 환경으로 사용한다.</content>

<sub>Data collection</sub>
<content>FinanceDataReader 라이브러리로 NETFLIX, 핀터레스트, 메타플렛폼스, 스포티파이의 2018년 1월 2일부터 2023년 12월 29일까지의 주가 데이터를 수집했다.</content>

<main>DATA PREPROCESSING</main>
<sub>Make derived variable</sub>
<content>Roc와 MA는 파생변수로 이전 시점을 포함하여 rolling되는 변수이므로 첫 시점은 NaN 상태이다. 따라서 2018년 데이터를 추가로 수집하여 파생 변수 생성에 활용할 필요가 있다.</content>

<sub>Add indecators</sub>
<content>금융 시계열 데이터에 대한 기술 분석을 도와주는 TA(Technical Analysis) Library이다.</content>

<sub>Peer Analysis</sub>
<content>한국투자증권에 따르면, PINS, FB, SPOT과 같은 기업들은 Netflix와 유사한 사업을 영위하고 있으며 해당 기간 종가를 feature로 추가하고 있다.</content>

<main>MODELING</main>
<sub>Time Series</sub>
<content>시간에 따른 데이터 패턴을

#Method2

In [ ]:
from langchain_openai import ChatOpenAI
import os

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    openai_api_key=os.environ['OPENAI_API_KEY']
)

In [ ]:
from langchain_openai import ChatOpenAI
import os

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    openai_api_key=os.environ['OPENAI_API_KEY']
)

def extract_information(text):
    prompt = f"""
    Classify the provided text into the following categories without paraphrasing, summarizing, altering, or omitting any part of the original text:
    - <subject>: Title of the document or project.
    - <team>: List of team members.
    - <index>: Main topics or sections in the document.
    - <main>: Main topic headings.
    - <sub>: Subtopics under each main topic.
    - <content>: Detailed content related to each subtopic.

    Ensure that all text is classified without omitting any content. Each classified section should be enclosed in the appropriate tags. Maintain the order of the text as it appears. If a text doesn't clearly fit into a category, classify it into the closest relevant category. Do not paraphrase, summarize, alter, or omit any part of the original text in any way. Ensure that no part of the text is omitted in the classification and tagging process, even if it seems unnecessary.

    First, classify and tag parts of the text that fit into the categories of <subject>, <team>, <index>, <main>, and <sub>. Then, wrap all remaining text as <content>.

    Example format:
    <subject>Netflix Stock Price Prediction with News Topic & Sentiment</subject>
    <team>12기 송규헌, 12기 권도영, 12기 이태경, 13기 김서윤, 13기 한진솔</team>
    <index>INTRODUCTION, DATA PREPROCESSING, MODELING, CONCLUSIONS AND LIMITATION</index>
    <main>웹툰 추천 시스템 구현</main>
    <sub>기존 사용자</sub>
    <content>사용자가 이전에 봤던 웹툰: 올가미(17회), 외모지상주의(16회), 이상한 변호사 우영우(7회). 타입 스토리: 장르 판타지, 드라마 등. 웹툰 추천 결과: rmse: 0.10001713</content>
    <sub>신규 사용자</sub>
    <content>좋아하는 웹툰 3개 입력: 이별 후 사내 결혼, 순정말고 순종, 다시 쓰는 연애사. 타입 스토리: 장르 로맨스 등. 웹툰 추천 결과: rmse: 0.10001718</content>
    <main>결론 및 향후 과제</main>
    <sub>프로젝트 요약 및 의의</sub>
    <content>CF, NCF, SVD, ALS 등 여러 가지 추천 알고리즘 모델을 구축하고 비교 분석한 결과를 토대로 최적의 모델을 선택함으로써, 사용자에게 더 나은 추천을 제공할 수 있음. Explicit feedback 없이 Implicit feedback을 사용하여 ALS 모델을 구축하고, 이를 통해 최상의 결과를 도출해냄. 추천 시스템을 통해 사용자의 선호에 맞는 웹툰을 제공함으로써 사용자들의 만족도를 향상시키고 플랫폼 이용률을 증가시킬 수 있음</content>
    <sub>한계점</sub>
    <content>이미 정제된 데이터를 사용한 점. 데이터 내 사용자와 아이템 간의 interaction 여부만 포함되어 있고, rating 값이 없어서 사용자의 선호도를 정확하게 파악하기 어려웠음. 추천 시스템은 사용자의 이전 행동을 기반으로 작동하기 때문에, interaction 데이터가 없는 신규 사용자에 대한 추천에는 적합하지 않을 수 있음 (Cold Start Problem)</content>
    <sub>향후과제</sub>
    <content>explicit feedback(명시적데이터) 수집을 통한 모델 성능 향상</content>

    Text: {text}
    """

    response = llm.invoke(prompt)
    extracted_info = response.content.strip()
    return extracted_info

extracted_info = extract_information(text)
print(extracted_info)


<subject>Netflix Stock Price Prediction with News Topic & Sentiment</subject>
<team>12기 송규헌, 12기 권도영, 12기 이태경, 13기 김서윤, 13기 한진솔</team>
<index>INTRODUCTION, DATA PREPROCESSING, MODELING, CONCLUSIONS AND LIMITATION</index>
<main>INTRODUCTION</main>
<sub>Background of topic selection</sub>
<content>뉴스가 주가 변동에 미치는 영향 탐구
주가 예측에 뉴스를 활용할 수 있는지 탐구해보고자 함
주가를 예측하는 데 사용하는 데이터로 뉴스의 감성분석 및 토픽 모델링 결과를 사용하고자 함
뉴스 기사에서는 주로 한 기업에 대해 보도하고 있어 예측 대상은 한 개의 주식 종목으로 정함
뉴스 감성 분석/토픽 모델링 결과를 활용하여 주가를 예측하는 프로젝트는 많지 않아, 직접 뉴스 데이터를 활용하고자 함</content>
<sub>Brief Project Introduction</sub>
<content>[목표]
뉴스 데이터를 활용한 주가 예측 모델의 최적화 및 효율적인 파라미터 선정
[주요 내용]
장기적인 추세 예측에 효과적인 모델 판별: LSTM vs. GRU vs. Transformer
실험을 통한 모델 파라미터 튜닝
뉴스 감성분석과 토픽 모델링의 주가 예측 유용성 검토
각 실험 결과 평가에 필요한 지표 선정
[개발 환경]
Colab</content>
<sub>Data collection</sub>
<content>FinanceDataReader 라이브러리를 활용하여 NETFLIX 기업의 2018년 1월 2일 ~ 2023년 12월 29일
주가 데이터 수집
동일한 기간에 대한 핀터레스트, 메타플렛폼스, 스포티파이의 주가 데이터 수집</content>
<main>DATA PREPROCESSING</main>
<sub>Make derived variab